# Importações de dados brutos

In [1]:
from util import *
meus_uteis()

import os

import joblib

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# from scipy import stats
from scipy.stats import mannwhitneyu, normaltest, ranksums, chisquare
from statsmodels.stats.weightstats import zconfint, DescrStatsW, ztest

-------------
-=< Lista >=-
-------------
0 -> printLinha()
1 -> printLis()
2 -> printDic()
3 -> timeProcess()
4 -> import_started()
5 -> img_save()
6 -> setup_img_save()
7 -> mask_corr_graphic()
8 -> structured_confusion()


In [2]:
df = pd.read_csv('./data/dados_completos.csv')
df.set_index(df.id, inplace=True)
del df["id"]
df.head()

,Data,IP,TCLE,Língua,Figura,Dintinguíveis,Proeminentes,Descrição,Eficiente,Útil,Estética,Fonte,Compreensão,Interesse,Grau,Área,Origem
id,,,,,,,,,,,,,,,,,
1,2020-03-02 13:31:20,200.134.9.2,s,pt,Reovírus Aviário,2,1,Sim,1,5,2,Sim,1,1,Doutorado,Exatas,Pesquisador
2,2020-03-02 17:45:41,191.52.71.228,s,pt,Reovírus Aviário,3,3,Sim,2,2,4,Não,4,2,Doutorado,Biológicas,Designer
3,2020-03-02 18:32:50,189.14.35.136,s,pt,Células Tronco,4,3,Sim,3,5,4,Não,3,3,Doutorado,Exatas,Pesquisador
4,2020-03-02 19:33:38,170.239.53.11,s,pt,Células Tronco,4,4,Sim,5,5,5,Sim,5,5,Mestrado,Biológicas,Designer
5,2020-03-03 11:50:20,189.103.158.150,s,pt,T. gondii,5,1,Sim,5,5,2,Sim,5,4,Doutorado,Biológicas,Pesquisador


In [3]:
colunas = df.columns
perguntas = {
    colunas[4 + 1]: '1) Na imagem, o quão claramente os elementos são distinguíveis?',
    colunas[4 + 2]: '2) O quão visualmente proeminentes são os elementos da imagem?',
    colunas[4 + 3]: '8) Existem elementos da imagem que requerem descrição adicional?',
    colunas[4 + 4]: '3) O quão eficiente são os elementos da imagem na representação dos dados?',
    colunas[4 + 5]: '4) O quão útil é a aplicação de uma imagem esquemática – desenho científico ou graphic abstract para representar os dados?',
    colunas[4 + 6]: '5) O quanto a imagem é esteticamente bem elaborada?',
    colunas[4 + 7]: '9) A imagem apresentada faz você experimentar uma nova fonte de dados?',
    colunas[4 + 8]: '6) Classifique o quanto a imagem facilitou a compreensão do conteúdo da informação.',
    colunas[4 + 9]: '7) Classifique o quanto a imagem despertou o seu interesse pelo conteúdo.',
    colunas[4 + 10]: 'Qual o seu grau de formação acadêmica?',
    colunas[4 + 11]: 'Área de formação?',
}
perguntas

{'Dintinguíveis': '1) Na imagem, o quão claramente os elementos são distinguíveis?',
 'Proeminentes': '2) O quão visualmente proeminentes são os elementos da imagem?',
 'Descrição': '8) Existem elementos da imagem que requerem descrição adicional?',
 'Eficiente': '3) O quão eficiente são os elementos da imagem na representação dos dados?',
 'Útil': '4) O quão útil é a aplicação de uma imagem esquemática – desenho científico ou graphic abstract para representar os dados?',
 'Estética': '5) O quanto a imagem é esteticamente bem elaborada?',
 'Fonte': '9) A imagem apresentada faz você experimentar uma nova fonte de dados?',
 'Compreensão': '6) Classifique o quanto a imagem facilitou a compreensão do conteúdo da informação.',
 'Interesse': '7) Classifique o quanto a imagem despertou o seu interesse pelo conteúdo.',
 'Grau': 'Qual o seu grau de formação acadêmica?',
 'Área': 'Área de formação?'}

# Geral

In [4]:
df.head()

,Data,IP,TCLE,Língua,Figura,Dintinguíveis,Proeminentes,Descrição,Eficiente,Útil,Estética,Fonte,Compreensão,Interesse,Grau,Área,Origem
id,,,,,,,,,,,,,,,,,
1,2020-03-02 13:31:20,200.134.9.2,s,pt,Reovírus Aviário,2,1,Sim,1,5,2,Sim,1,1,Doutorado,Exatas,Pesquisador
2,2020-03-02 17:45:41,191.52.71.228,s,pt,Reovírus Aviário,3,3,Sim,2,2,4,Não,4,2,Doutorado,Biológicas,Designer
3,2020-03-02 18:32:50,189.14.35.136,s,pt,Células Tronco,4,3,Sim,3,5,4,Não,3,3,Doutorado,Exatas,Pesquisador
4,2020-03-02 19:33:38,170.239.53.11,s,pt,Células Tronco,4,4,Sim,5,5,5,Sim,5,5,Mestrado,Biológicas,Designer
5,2020-03-03 11:50:20,189.103.158.150,s,pt,T. gondii,5,1,Sim,5,5,2,Sim,5,4,Doutorado,Biológicas,Pesquisador


In [5]:
df.describe()

,Dintinguíveis,Proeminentes,Eficiente,Útil,Estética,Compreensão,Interesse
count,472.000000,472.000000,472.000000,472.000000,472.000000,472.000000,472.000000
mean,3.786017,3.576271,3.552966,4.614407,3.567797,3.599576,3.489407
std,1.227245,1.274654,1.247227,0.909751,1.340082,1.314075,1.376129
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,3.000000,3.000000,3.000000,5.000000,3.000000,3.000000,2.000000
50%,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000
75%,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 472 entries, 1 to 472
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Data           472 non-null    object
 1   IP             472 non-null    object
 2   TCLE           472 non-null    object
 3   Língua         472 non-null    object
 4   Figura         472 non-null    object
 5   Dintinguíveis  472 non-null    int64 
 6   Proeminentes   472 non-null    int64 
 7   Descrição      472 non-null    object
 8   Eficiente      472 non-null    int64 
 9   Útil           472 non-null    int64 
 10  Estética       472 non-null    int64 
 11  Fonte          472 non-null    object
 12  Compreensão    472 non-null    int64 
 13  Interesse      472 non-null    int64 
 14  Grau           472 non-null    object
 15  Área           472 non-null    object
 16  Origem         472 non-null    object
dtypes: int64(7), object(10)
memory usage: 66.4+ KB


# Agrupando

## Analisando a pergunta Descrição

### Geral

In [7]:
colunas_perguntas = df.columns[[7,11,14,15,16]]
df_quantitativo = df[colunas_perguntas]
df_quantitativo.head()

,Descrição,Fonte,Grau,Área,Origem
id,,,,,
1,Sim,Sim,Doutorado,Exatas,Pesquisador
2,Sim,Não,Doutorado,Biológicas,Designer
3,Sim,Não,Doutorado,Exatas,Pesquisador
4,Sim,Sim,Mestrado,Biológicas,Designer
5,Sim,Sim,Doutorado,Biológicas,Pesquisador


In [8]:
df_descricao = df_quantitativo[colunas_perguntas[[0,4]]].value_counts(normalize=False)
df_descricao

Descrição  Origem     
Sim        Pesquisador    206
           Designer       168
Não        Designer        68
           Pesquisador     30
dtype: int64

In [9]:
dic_descricao = {
    'Designer': [
                    df_descricao.loc['Sim',:].loc['Designer'],
                    df_descricao.loc['Não',:].loc['Designer'],
                 ],
    'Pesquisador': [
                      df_descricao.loc['Sim',:].loc['Pesquisador'],
                      df_descricao.loc['Não',:].loc['Pesquisador'],
                    ],
}
dic_descricao

{'Designer': [168, 68], 'Pesquisador': [206, 30]}

In [10]:
index=['Sim','Não']
df_descricao_compila = pd.DataFrame(data=np.nan, index=index, columns=dic_descricao.keys())
df_descricao_compila['Pesquisador'] = dic_descricao['Pesquisador']
df_descricao_compila['Designer'] = dic_descricao['Designer']
df_descricao_compila

,Designer,Pesquisador
Sim,168,206
Não,68,30


In [11]:
frequencias = [[df_descricao_compila.loc['Sim','Pesquisador'],df_descricao_compila.loc['Não','Pesquisador']],
              [df_descricao_compila.loc['Sim','Designer'],df_descricao_compila.loc['Não','Designer']]]
frequencias

[[206, 30], [168, 68]]

In [12]:
_, p_valor = chisquare(f_exp=df_descricao_compila['Pesquisador'], f_obs=df_descricao_compila['Designer'])
if p_valor < .001:
    p_valor = '<0.001'

df_descricao_compila['p-valor'] = (p_valor, np.nan)
# df_descricao_compila.to_csv('df_descricao_compila.csv')
df_descricao_compila

,Designer,Pesquisador,p-valor
Sim,168,206,<0.001
Não,68,30,NaN


### Área

#### Biológicas

In [13]:
def teste_estatistico(frequencias):
    diff = sum(frequencias[1]) - sum(frequencias[0])
    
    if diff != 0 :
        frequencias[1][0] = frequencias[1][0]-diff

    try:
        f_exp = frequencias[1]
        f_obs = frequencias[0]
        _, p_valor = chisquare(f_exp=f_exp, f_obs=f_obs)
    except:
        p_valor = 0.1
        
    if p_valor < .001:
        p_valor = '<0.001'
    else:
        p_valor = str(round(p_valor, 3))
    return p_valor

In [14]:
def tabela_freq_qualitattivo(df_teste):
    df_fonte = df_teste.value_counts(normalize=False)
    dic_fonte = {
      'Designer': [
                      df_fonte.loc['Sim',:].loc['Designer'],
                      df_fonte.loc['Não',:].loc['Designer'],
                  ],
      'Pesquisador': [
                        df_fonte.loc['Sim',:].loc['Pesquisador'],
                        df_fonte.loc['Não',:].loc['Pesquisador'],
                      ],
    }
    index=['Sim','Não']
    df_fonte_compila = pd.DataFrame(data=np.nan, index=index, columns=dic_fonte.keys())
    df_fonte_compila['Pesquisador'] = dic_fonte['Pesquisador']
    df_fonte_compila['Designer'] = dic_fonte['Designer']
    df_fonte_compila = df_fonte_compila[df_fonte_compila.columns[[1,0]]]
    frequencias = [[df_fonte_compila.loc['Sim','Pesquisador'],df_fonte_compila.loc['Não','Pesquisador']],
                [df_fonte_compila.loc['Sim','Designer'],df_fonte_compila.loc['Não','Designer']]]
    
    p_valor = teste_estatistico(frequencias)
    
    df_fonte_compila['p-valor'] = (p_valor, np.nan)
    return df_fonte_compila

In [15]:
df_descricao_compila_bio = df_quantitativo[df_quantitativo["Área"] == "Biológicas"][colunas_perguntas[[0,4]]]
df_descricao_compila_bio.value_counts()

Descrição  Origem     
Sim        Pesquisador    161
           Designer       130
Não        Designer        48
           Pesquisador     18
dtype: int64

In [16]:
df_descricao_compila_bio = tabela_freq_qualitattivo(df_descricao_compila_bio)
# df_descricao_compila_bio.to_csv("df_descricao_compila_bio.csv")
df_descricao_compila_bio

,Pesquisador,Designer,p-valor
Sim,161,130,<0.001
Não,18,48,NaN


#### Exatas

In [17]:
df_descricao_compila_exatas = df_quantitativo[df_quantitativo["Área"] == "Exatas"][colunas_perguntas[[0,4]]]
df_descricao_compila_exatas.value_counts()

Descrição  Origem     
Sim        Designer       23
           Pesquisador    20
Não        Designer       11
           Pesquisador     3
dtype: int64

In [18]:
df_descricao_compila_exatas = tabela_freq_qualitattivo(df_descricao_compila_exatas)
# df_descricao_compila_exatas.to_csv("df_descricao_compila_exatas.csv")
df_descricao_compila_exatas

,Pesquisador,Designer,p-valor
Sim,20,23,<0.001
Não,3,11,NaN


#### Humanas

In [19]:
df_descricao_compila_humanas = df_quantitativo[df_quantitativo["Área"] == "Humanas"][colunas_perguntas[[0,4]]]
df_descricao_compila_humanas.value_counts()

Descrição  Origem     
Sim        Pesquisador    25
           Designer       15
Não        Designer        9
           Pesquisador     9
dtype: int64

In [20]:
df_descricao_compila_humanas = tabela_freq_qualitattivo(df_descricao_compila_humanas)
# df_descricao_compila_humanas.to_csv("df_descricao_compila_humanas.csv")
df_descricao_compila_humanas

,Pesquisador,Designer,p-valor
Sim,25,15,1.0
Não,9,9,NaN


### Grau

#### Graduação

In [21]:
df_descricao_compila_gradu = df_quantitativo[df_quantitativo["Grau"] == "Graduação"][colunas_perguntas[[0,4]]]
df_descricao_compila_gradu.value_counts()

Descrição  Origem     
Sim        Designer       18
           Pesquisador    17
Não        Designer        8
           Pesquisador     2
dtype: int64

In [22]:
df_descricao_compila_gradu = tabela_freq_qualitattivo(df_descricao_compila_gradu)
# df_descricao_compila_gradu.to_csv("df_descricao_compila_gradu.csv")
df_descricao_compila_gradu

,Pesquisador,Designer,p-valor
Sim,17,18,0.005
Não,2,8,NaN


#### Pós-Graduação

In [23]:
df_descricao_compila_posGradu = df_quantitativo[df_quantitativo["Grau"] == "Pós-Graduação"][colunas_perguntas[[0,4]]]
df_descricao_compila_posGradu.value_counts()

Descrição  Origem     
Sim        Designer       33
           Pesquisador    31
Não        Designer       14
           Pesquisador     7
dtype: int64

In [24]:
df_descricao_compila_posGradu = tabela_freq_qualitattivo(df_descricao_compila_posGradu)
# df_descricao_compila_posGradu.to_csv("df_descricao_compila_posGradu.csv")
df_descricao_compila_posGradu

,Pesquisador,Designer,p-valor
Sim,31,33,0.019
Não,7,14,NaN


#### Mestrado

In [25]:
df_descricao_compila_mestrado = df_quantitativo[df_quantitativo["Grau"] == "Mestrado"][colunas_perguntas[[0,4]]]
df_descricao_compila_mestrado.value_counts()

Descrição  Origem     
Sim        Pesquisador    89
           Designer       62
Não        Designer       23
           Pesquisador    13
dtype: int64

In [26]:
df_descricao_compila_mestrado = tabela_freq_qualitattivo(df_descricao_compila_mestrado)
# df_descricao_compila_mestrado.to_csv("df_descricao_compila_mestrado.csv")
df_descricao_compila_mestrado

,Pesquisador,Designer,p-valor
Sim,89,62,0.018
Não,13,23,NaN


#### Doutorado

In [27]:
df_descricao_compila_doc = df_quantitativo[df_quantitativo["Grau"] == "Doutorado"][colunas_perguntas[[0,4]]]
df_descricao_compila_doc.value_counts()

Descrição  Origem     
Sim        Pesquisador    69
           Designer       55
Não        Designer       23
           Pesquisador     8
dtype: int64

In [28]:
df_descricao_compila_doc = tabela_freq_qualitattivo(df_descricao_compila_doc)
# df_descricao_compila_doc.to_csv("df_descricao_compila_doc.csv")
df_descricao_compila_doc

,Pesquisador,Designer,p-valor
Sim,69,55,<0.001
Não,8,23,NaN


## Analisando a pergunta Fonte

### Geral

In [29]:
df_fonte_compila = df_quantitativo[colunas_perguntas[[1,4]]]
df_fonte_compila.value_counts()

Fonte  Origem     
Sim    Designer       191
       Pesquisador    146
Não    Pesquisador     90
       Designer        45
dtype: int64

In [30]:
df_fonte_compila = tabela_freq_qualitattivo(df_fonte_compila)
# df_fonte_compila.to_csv("df_fonte_compila.csv")
df_fonte_compila

,Pesquisador,Designer,p-valor
Sim,146,191,<0.001
Não,90,45,NaN


### Área

#### Bilógicas

In [31]:
df_fonte_compila_bio = df_quantitativo[df_quantitativo["Área"] == "Biológicas"][colunas_perguntas[[1,4]]]
df_fonte_compila_bio.value_counts()

Fonte  Origem     
Sim    Designer       140
       Pesquisador    104
Não    Pesquisador     75
       Designer        38
dtype: int64

In [32]:
df_fonte_compila_bio = tabela_freq_qualitattivo(df_fonte_compila_bio)
# df_fonte_compila_bio.to_csv("df_fonte_compila_bio.csv")
df_fonte_compila_bio

,Pesquisador,Designer,p-valor
Sim,104,140,<0.001
Não,75,38,NaN


#### Exatas

In [33]:
df_fonte_compila_exatas = df_quantitativo[df_quantitativo["Área"] == "Exatas"][colunas_perguntas[[1,4]]]
df_fonte_compila_exatas.value_counts()

Fonte  Origem     
Sim    Designer       30
       Pesquisador    14
Não    Pesquisador     9
       Designer        4
dtype: int64

In [34]:
df_fonte_compila_exatas = tabela_freq_qualitattivo(df_fonte_compila_exatas)
# df_fonte_compila_exatas.to_csv("df_fonte_compila_exatas.csv")
df_fonte_compila_exatas

,Pesquisador,Designer,p-valor
Sim,14,30,0.006
Não,9,4,NaN


#### Humanas

In [35]:
df_fonte_compila_humanas = df_quantitativo[df_quantitativo["Área"] == "Humanas"][colunas_perguntas[[1,4]]]
df_fonte_compila_humanas.value_counts()

Fonte  Origem     
Sim    Pesquisador    28
       Designer       21
Não    Pesquisador     6
       Designer        3
dtype: int64

In [36]:
df_fonte_compila_humanas = tabela_freq_qualitattivo(df_fonte_compila_humanas)
# df_fonte_compila_humanas.to_csv("df_fonte_compila_humanas.csv")
df_fonte_compila_humanas

,Pesquisador,Designer,p-valor
Sim,28,21,0.07
Não,6,3,NaN


### Grau

#### Graduação

In [37]:
df_fonte_compila_gradua = df_quantitativo[df_quantitativo["Grau"] == "Graduação"][colunas_perguntas[[1,4]]]
df_fonte_compila_gradua.value_counts()

Fonte  Origem     
Sim    Designer       22
       Pesquisador    10
Não    Pesquisador     9
       Designer        4
dtype: int64

In [38]:
df_fonte_compila_gradua = tabela_freq_qualitattivo(df_fonte_compila_gradua)
# df_fonte_compila_gradua.to_csv("df_fonte_compila_gradua.csv")
df_fonte_compila_gradua

,Pesquisador,Designer,p-valor
Sim,10,22,0.005
Não,9,4,NaN


#### Pós-Graduação

In [39]:
df_fonte_compila_posGradua = df_quantitativo[df_quantitativo["Grau"] == "Pós-Graduação"][colunas_perguntas[[1,4]]]
df_fonte_compila_posGradua.value_counts()

Fonte  Origem     
Sim    Designer       37
       Pesquisador    27
Não    Pesquisador    11
       Designer       10
dtype: int64

In [40]:
df_fonte_compila_posGradua = tabela_freq_qualitattivo(df_fonte_compila_posGradua)
# df_fonte_compila_posGradua.to_csv("df_fonte_compila_posGradua.csv")
df_fonte_compila_posGradua

,Pesquisador,Designer,p-valor
Sim,27,37,0.713
Não,11,10,NaN


#### Mestrado

In [41]:
df_fonte_compila_mestrado = df_quantitativo[df_quantitativo["Grau"] == "Mestrado"][colunas_perguntas[[1,4]]]
df_fonte_compila_mestrado.value_counts()

Fonte  Origem     
Sim    Designer       73
       Pesquisador    59
Não    Pesquisador    43
       Designer       12
dtype: int64

In [42]:
df_fonte_compila_mestrado = tabela_freq_qualitattivo(df_fonte_compila_mestrado)
# df_fonte_compila_mestrado.to_csv("df_fonte_compila_mestrado.csv")
df_fonte_compila_mestrado

,Pesquisador,Designer,p-valor
Sim,59,73,<0.001
Não,43,12,NaN


#### Doutorado

In [43]:
df_fonte_compila_doc = df_quantitativo[df_quantitativo["Grau"] == "Doutorado"][colunas_perguntas[[1,4]]]
df_fonte_compila_doc.value_counts()

Fonte  Origem     
Sim    Designer       59
       Pesquisador    50
Não    Pesquisador    27
       Designer       19
dtype: int64

In [44]:
df_fonte_compila_doc = tabela_freq_qualitattivo(df_fonte_compila_doc)
# df_fonte_compila_doc.to_csv("df_fonte_compila_doc.csv")
df_fonte_compila_doc

,Pesquisador,Designer,p-valor
Sim,50,59,0.034
Não,27,19,NaN
